In [4]:
# !python -m pip install emoji --upgrade

  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49721 sha256=9cd96555cd3d6a03e39369b00b9deaa63abf0270852c953dc50c1a7999ca4125
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\4e\bf\6b\2e22b3708d14bf6384f862db539b044d6931bd6b14ad3c9adc
Successfully built emoji


In [3]:
import json
from main import preprocess, make_meta
from tokenization import tokenize
from lemmatization import lemmatize

In [2]:
meta = make_meta()
len(meta)

192

### Пример использования на 1 вхождении

In [4]:
test = '[id00000|имя], какая у вас ох*енная собака! 😍💤😥'
tok = tokenize(test)

for i, params in meta.items():
    print(preprocess(tok, params=params))

username , какая у вас охёенная собака ! 😍 💤 😥
username , какая у вас vulgar собака ! 😍 💤 😥
username , какая у вас охёенная собака ! 
username , какая у вас vulgar собака !
username , какая у вас охёенная собака ! emoji emoji emoji
username , какая у вас vulgar собака ! emoji emoji emoji
username , какая у вас охёенная собака ! pos_emoji neutral_emoji neg_emoji
username , какая у вас vulgar собака ! pos_emoji neutral_emoji neg_emoji
username , охёенная собака ! 😍 💤 😥
username , vulgar собака ! 😍 💤 😥
username , охёенная собака ! 
username , vulgar собака !
username , охёенная собака ! emoji emoji emoji
username , vulgar собака ! emoji emoji emoji
username , охёенная собака ! pos_emoji neutral_emoji neg_emoji
username , vulgar собака ! pos_emoji neutral_emoji neg_emoji
username , какой у вы охёенный собака ! 😍 💤 😥
username , какой у вы vulgar собака ! 😍 💤 😥
username , какой у вы охёенный собака ! 
username , какой у вы vulgar собака !
username , какой у вы охёенный собака ! emoji emoji e

username какой у вы охёенный собака 
username какой у вы vulgar собака
username какой у вы охёенный собака emoji emoji emoji
username какой у вы vulgar собака emoji emoji emoji
username какой у вы охёенный собака pos_emoji neutral_emoji neg_emoji
username какой у вы vulgar собака pos_emoji neutral_emoji neg_emoji
username охёенный собака 😍 💤 😥
username vulgar собака 😍 💤 😥
username охёенный собака 
username vulgar собака
username охёенный собака emoji emoji emoji
username vulgar собака emoji emoji emoji
username охёенный собака pos_emoji neutral_emoji neg_emoji
username vulgar собака pos_emoji neutral_emoji neg_emoji


## Пример использования на всех данных

In [6]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

In [7]:
from merge_df import table_to_dataframe

In [27]:
filepath = '../Data/Test_data.xlsx'
raw_df = table_to_dataframe(filepath)
raw_df.dropna(axis=0, how='any', inplace=True)
raw_df.drop(columns=['index'], inplace=True)
len(raw_df)

4436

In [32]:
df = raw_df[raw_df['разметка'] != 'skip'].copy()
df['разметка'].value_counts()

negative    436
positive    367
neutral     321
speech      189
Name: разметка, dtype: int64

In [33]:
print('итого вхождений:', len(df))

итого вхождений: 1313


In [34]:
df['tokenized'] = df['текст'].progress_apply(tokenize)
df['lemmatized'] = df['tokenized'].progress_apply(lemmatize)

In [35]:
df.head()

,разметка,текст,контекст,community_id,community_type,tokenized,lemmatized
0,speech,Спасибо!,post,1,nontoxic,спасибо !,спасибо !
1,speech,вау спасибо большое!!,post,1,nontoxic,вау спасибо большое ! !,вау спасибо большой ! !
2,speech,Супер! Благодарю🙏,post,1,nontoxic,супер ! благодарю 🙏,супер ! благодарить 🙏
3,speech,"офигеть, спасибо!",post,1,nontoxic,"офигеть , спасибо !","офигеть , спасибо !"
4,speech,спасибо за проделанную работу 💙,post,1,nontoxic,спасибо за проделанную работу 💙,спасибо за проделать работа 💙


In [36]:
for i, params in tqdm(meta.items()):
    if params['lemmatization'] == 'no':
        df[i] = [preprocess(tok, params=params) for tok in df['tokenized']]
    if params['lemmatization'] == 'yes':
        params['lemmatization'] = 'no'
        df[i] = [preprocess(tok, params=params) for tok in df['lemmatized']]

In [37]:
df.sample(5)

,разметка,текст,контекст,community_id,community_type,tokenized,lemmatized,0,1,2,...,182,183,184,185,186,187,188,189,190,191
1007,neutral,Фига булавка здоровая,post,2,nontoxic,фига булавка здоровая,фига булавка здоровый,фига булавка здоровая,фига булавка здоровая,фига булавка здоровая,...,фига булавка здоровый,фига булавка здоровый,фига булавка здоровый,фига булавка здоровый,фига булавка здоровый,фига булавка здоровый,фига булавка здоровый,фига булавка здоровый,фига булавка здоровый,фига булавка здоровый
1034,positive,"Вау, какая интересная внешность.",post,2,nontoxic,"вау , какая интересная внешность .","вау , какой интересный внешность .","вау , какая интересная внешность .","вау , какая интересная внешность .","вау , какая интересная внешность .",...,вау какой интересный внешность,вау какой интересный внешность,вау интересный внешность,вау интересный внешность,вау интересный внешность,вау интересный внешность,вау интересный внешность,вау интересный внешность,вау интересный внешность,вау интересный внешность
2982,neutral,"[id113727070|Олег], в основном.. фильмы ужасов...","чот я такое же слышал про некий слешер, где уб...",3,toxic,"username , в основном . . фильмы ужасов имеют ...","username , в основное . . фильм ужас иметь воз...","username , в основном . . фильмы ужасов имеют ...","username , в основном . . фильмы ужасов имеют ...","username , в основном . . фильмы ужасов имеют ...",...,username в основное фильм ужас иметь возрастно...,username в основное фильм ужас иметь возрастно...,username основное фильм ужас иметь возрастной ...,username основное фильм ужас иметь возрастной ...,username основное фильм ужас иметь возрастной ...,username основное фильм ужас иметь возрастной ...,username основное фильм ужас иметь возрастной ...,username основное фильм ужас иметь возрастной ...,username основное фильм ужас иметь возрастной ...,username основное фильм ужас иметь возрастной ...
597,positive,"[id5098043|Елена], вау это один из лучших пабл...",[club167639293|альфия творит] !!! Она учится в...,2,nontoxic,"username , вау это один из лучших пабликов кот...","username , вау это один из хороший пабликовый ...","username , вау это один из лучших пабликов кот...","username , вау это один из лучших пабликов кот...","username , вау это один из лучших пабликов кот...",...,username вау это один из хороший пабликовый ко...,username вау это один из хороший пабликовый ко...,username вау хороший пабликовый видеть шикарны...,username вау хороший пабликовый видеть шикарны...,username вау хороший пабликовый видеть шикарны...,username вау хороший пабликовый видеть шикарны...,username вау хороший пабликовый видеть шикарны...,username вау хороший пабликовый видеть шикарны...,username вау хороший пабликовый видеть шикарны...,username вау хороший пабликовый видеть шикарны...
6767,positive,От души поржал с коментов😄,"Молодец, что взяла себя в руки и поняла, что к...",4,toxic,от души поржал с коментов 😄,от душа поржать с комент 😄,от души поржал с коментов 😄,от души поржал с коментов 😄,от души поржал с коментов,...,от душа поржать с комент pos_emoji,от душа поржать с комент pos_emoji,душа поржать комент 😄,душа поржать комент 😄,душа поржать комент,душа поржать комент,душа поржать комент emoji,душа поржать комент emoji,душа поржать комент pos_emoji,душа поржать комент pos_emoji


In [42]:
df.to_csv('../Data/Test_data_preprocessed.tsv', sep='\t')

## Если нужно поменять опции то допишите дополнительные циклы main.py

In [3]:
with open('meta.json', 'w') as f:
    json.dump(meta, f, ensure_ascii=False)